In [1]:
from ipywidgets.widgets import Label, FloatProgress, FloatSlider, Button, Dropdown, FileUpload, Image, Layout, HBox, VBox, Box, Output, IntProgress, IntSlider
from IPython.display import display, clear_output
import numpy as np
from numpy import dot, exp, mgrid, pi, ravel, square, uint8, zeros
import math
import bqplot as bq
import time
import threading
from copy import deepcopy

In [2]:
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import os
import io
from itertools import product
import PIL.Image as ImagePIL
from array import array

In [4]:
def medianFilter(img, progress):
    img_noisy1 = img
    m, n = img_noisy1.shape[0], img_noisy1.shape[1]
    img_new1 = np.zeros([m, n])

    for i in range(1, m-1):
        progress.value += int((i+1)/100)
        for j in range(1, n-1):
            temp = [img_noisy1[i-1, j-1],
                   img_noisy1[i-1, j],
                   img_noisy1[i-1, j + 1],
                   img_noisy1[i, j-1],
                   img_noisy1[i, j],
                   img_noisy1[i, j + 1],
                   img_noisy1[i + 1, j-1],
                   img_noisy1[i + 1, j],
                   img_noisy1[i + 1, j + 1]]

            temp = sorted(temp)
            img_new1[i, j]= temp[4] # Median => Sorted

    img_new1 = img_new1.astype(np.uint8)
    cv2.imwrite('medianFilter.jpg', img_new1)

In [5]:
def gen_gaussian_kernel(k_size, sigma):
    center = k_size // 2
    x, y = mgrid[0 - center : k_size - center, 0 - center : k_size - center]
    g = 1 / (2 * pi * sigma) * exp(-(square(x) + square(y)) / (2 * square(sigma)))
    return g


def gaussianFilter(image, k_size, sigma, progress):
    height, width = image.shape[0], image.shape[1]
    dst_height = height - k_size + 1
    dst_width = width - k_size + 1

    image_array = zeros((dst_height * dst_width, k_size * k_size))
    row = 0
    for i, j in product(range(dst_height), range(dst_width)):
        progress.value += int((row+1)/100)
        window = ravel(image[i : i + k_size, j : j + k_size])
        image_array[row, :] = window
        row += 1

    gaussian_kernel = gen_gaussian_kernel(k_size, sigma)
    filter_array = ravel(gaussian_kernel)

    dst = dot(image_array, filter_array).reshape(dst_height, dst_width).astype(uint8)

    cv2.imwrite('gaussianFilter.jpg', dst)

In [6]:
def averaging(img, progress):
    filtr = (1/16) * np.matrix([[1,2,1],[2,4,2],[1,2,1]], dtype=int)
    n_r, n_c, ch = img.shape
    g = np.zeros([n_r, n_c])
    for r in range(1, n_r-1):
        progress.value += int((r+1)/100)
        for c in range(1, n_c-1):
            h = img[r-1, c-1] * filtr[0, 0] + img[r-1, c] * filtr[0, 1] + img[r-1, c + 1] * filtr[0, 2] + img[r, c-1] * filtr[1, 0] + img[r, c] * filtr[1, 1] + img[r, c + 1] * filtr[1, 2] + img[r + 1, c-1] * filtr[2, 0] + img[r + 1, c] * filtr[2, 1] + img[r + 1, c + 1] * filtr[2, 2]

            g[r, c]= h

    g = g.astype(np.uint8)
    cv2.imwrite('averaging.jpg', g)

In [7]:
def unsharpMaskingAndHighboostFiltering(img, k, progress):
    img = img.reshape(img.shape[0], img.shape[1], 1)
    blur = cv2.GaussianBlur(img, (5,5), 0)
    blur = blur.reshape(blur.shape[0], blur.shape[1], 1)
    mask = img - blur
    final = img + k * mask
    progress.value = 100
    cv2.imwrite('unsharpMaskingAndHighboostFiltering.jpg', final)
# Highboost filtering if k > 1

In [8]:
def sobelOperator(test, progress):
    image_file = test
    input_image = test
    [nx, ny, nz] = np.shape(input_image)

    r_img, g_img, b_img = input_image[:, :, 0], input_image[:, :, 0], input_image[:, :, 0] # 0,1,2


    gamma = 1.400  
    r_const, g_const, b_const = 0.2126, 0.7152, 0.0722  
    grayscale_image = r_const * r_img ** gamma + g_const * g_img ** gamma + b_const * b_img ** gamma


    Gx = np.array([[1.0, 0.0, -1.0], [2.0, 0.0, -2.0], [1.0, 0.0, -1.0]])
    Gy = np.array([[1.0, 2.0, 1.0], [0.0, 0.0, 0.0], [-1.0, -2.0, -1.0]])
    [rows, columns] = np.shape(grayscale_image)  
    sobel_filtered_image = np.zeros(shape=(rows, columns))  


    for i in range(rows - 2):
        progress.value += int((i+1)/100)
        for j in range(columns - 2):
            gx = np.sum(np.multiply(Gx, grayscale_image[i:i + 3, j:j + 3]))  
            gy = np.sum(np.multiply(Gy, grayscale_image[i:i + 3, j:j + 3]))  
            sobel_filtered_image[i + 1, j + 1] = np.sqrt(gx ** 2 + gy ** 2)  

    cv2.imwrite('sobelOperator.jpg', sobel_filtered_image)

In [9]:
def saltAndPepperNoise(img, progress):
    row, col, ch = img.shape
    
    number_of_white_pixels = random.randint(100, 1000)
    for i in range(number_of_white_pixels):
        progress.value += int((i+1)/50)
        y=random.randint(1, row - 2)
        x=random.randint(1, col - 2)
        img[y][x] = 255
        img[y + 1][x] = 255
        img[y - 1][x] = 255
        #img[y + 1][x + 1] = 255
        #img[y - 1][x - 1] = 255
        img[y][x + 1] = 255
        img[y][x - 1] = 255
        #img[y + 1][x + 1] = 255
        #img[y - 1][x - 1] = 255
         
            
    number_of_black_pixels = random.randint(100, 1000)
    for i in range(number_of_black_pixels):
        progress.value += int((i+1)/50)
        y=random.randint(1, row - 2)
        x=random.randint(1, col - 2)
        img[y][x] = 0
        img[y + 1][x] = 0
        img[y - 1][x] = 0
        #img[y + 1][x + 1] = 0
        #img[y - 1][x - 1] = 0
        img[y][x + 1] = 0
        img[y][x - 1] = 0
        #img[y + 1][x + 1] = 0
        #img[y - 1][x - 1] = 0
         
    cv2.imwrite('saltAndPepperNoise.jpg', img)

#img = cv2.imread('pic2.png')
#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#plt.imshow(img, 'gray')
#plt.imshow(saltAndPepperNoise(img))

In [10]:
# loc = Mean (“centre”) of the distribution.
# scale = Standard deviation (spread or “width”) of the distribution. Must be non-negative.
def gaussianNoise(real_image, progress):
    gaussian = np.random.normal(loc=0, scale=1, size=real_image.size)
    gaussian = gaussian.reshape(real_image.shape[0], real_image.shape[1], real_image.shape[2]).astype('uint8')
    noisy_image = cv2.add(real_image, gaussian)
    cv2.imwrite('gaussianNoise.jpg', noisy_image)
    progress.value = 100

#img = cv2.imread('pic2.png', 0)
#plt.imshow(img, 'gray')
#plt.imshow(gaussianNoise(img), 'gray')

In [11]:
def imhist(im):
    m, n = im.shape
    h = [0.0] * 256
    for i in range(m):
        for j in range(n):
            h[im[i, j]]+=1
    return np.array(h)/(m*n)

def cumsum(h):
    return [sum(h[:i+1]) for i in range(len(h))]

def histeq(im, progress):
    #calculate Histogram
    h = imhist(im)
    cdf = np.array(cumsum(h))
    sk = np.uint8(255 * cdf)
    s1, s2 = im.shape
    Y = np.zeros_like(im)

    for i in range(0, s1):
        progress.value += int((i+1)/100)
        for j in range(0, s2):
            Y[i, j] = sk[im[i, j]]
    H = imhist(Y)

    return Y, h, H, sk

def histogramEq(test, progress):
    test = test.reshape(test.shape[0], test.shape[1], 1)
    img = np.uint8(test * 255.0)

    img = np.uint8((0.2126 * img[:,:,0]) + np.uint8(0.7152 * img[:,:,0]) + np.uint8(0.0722 * img[:,:,0])) # 0,1,2
    new_img, h, new_h, sk = histeq(img, progress)
    
    fig = plt.figure()
    fig.add_subplot(221)
    plt.plot(h)
    plt.title('Original histogram') # original histogram

    fig.add_subplot(222)
    plt.plot(new_h)
    plt.title('New histogram') #hist of eqlauized image

    fig.add_subplot(223)
    plt.plot(sk)
    plt.title('Transfer function') #transfer function
    
    cv2.imwrite('histogramEq.jpg', new_img)

In [12]:
def fourierTransform(real_image, progress):
    real_image = cv2.resize(real_image, (128, 128))
    M = real_image.shape[0] - 1
    N = real_image.shape[1] - 1
    dft_image = np.zeros((M, N), complex)
    Mu = 1
    v = 1
    j = 1
    for m in range(M):
        progress.value += int((m+1)/100)
        for n in range(N):
            for x in range(M):
                for y in range(N):
                    exp_power = -j * 2 * math.pi * ((Mu * x)/M + (v * y)/N)
                    dft_image[m][n] += real_image[x][y] * np.exp(exp_power)
    #return dft_image
    cv2.imwrite('fourierTransform.jpg', dft_image)

In [13]:
def kernel(x,a):
    if(np.abs(x)>=0)&(np.abs(x)<=1):
        res=(a+2)*(np.abs(x)**3)-(a+3)*(np.abs(x)**2)+1
        return res
    if(np.abs(x)>1)&(np.abs(x)<=2):
        res=a*(np.abs(x)**3)-(a*5)*(np.abs(x)**2)+((a*8)*np.abs(x))-a*4
        return res
    return 0

def pad(imag,col,r,c):

    pad=np.zeros((col+4,r+4,c))
    pad[2:col+2,2:r+2,:c]=imag
    
    pad[2:col+2,0:2,:c]=imag[:,0:1,:c]
    pad[2:col+2,r+2:r+4,:]=imag[:,r-1:r,:]
    
    pad[0:2,2:r+2,:c]=imag[0:1,:,:c]
    pad[col+2:col+4,2:r+2,:]=imag[col-1:col,:,:]
    
    pad[0:2,0:2,:c]=imag[0,0,:c]
    pad[0:2,r+2:r+4,:c]=imag[0,r-1,:c]
    pad[col+2:col+4,0:2,:c]=imag[col-1,0,:c]
    pad[col+2:col+4,r+2:r+4,:c]=imag[col-1,r-1,:c]
    
    return pad

def bicubicInterpolation(img, ratio, a, progress):
    img = img.reshape(img.shape[0], img.shape[1], 1)
    col, rows, ch =img.shape
    img= pad(img,col,rows,ch)
    
    dcol= math.floor(col*ratio)
    drows=math.floor(rows*ratio)

    matriximage=np.zeros((dcol,drows,1))
    v= 1/ratio
    
    for c in range(ch):
        for column in range(dcol):
            progress.value += int((column + 1)/100)
            for row in range(drows):

                x=row * v + 2
                y=column * v + 2
                
                
                x1= 1+x-math.floor(x)
                y1=1+y-math.floor(y)
                x2=x-math.floor(x)
                y2=y-math.floor(y)
                x3=math.floor(x)+1-x
                y3=math.floor(y)+1-y
                x4=math.floor(x)+2-x
                y4=math.floor(y)+2-y
                
                mat_kernelx=np.matrix([[kernel(x1,a),kernel(x2,a),kernel(x3,a),kernel(x4,a)]])
                
                mat_near=np.matrix([[img[int(y-y1),int(x-x1),c],img[int(y-y2),int(x-x1),c],img[int(y+y3),int(x-x1),c],img[int(y+y4),int(x-x1),c]],
                                    [img[int(y-y1),int(x-x2),c],img[int(y-y2),int(x-x2),c],img[int(y+y3),int(x-x2),c],img[int(y+y4),int(x-x2),c]],
                                    [img[int(y-y1),int(x+x3),c],img[int(y-y2),int(x+x3),c],img[int(y+y3),int(x+x3),c],img[int(y+y4),int(x+x3),c]],
                                    [img[int(y-y1),int(x+x4),c],img[int(y-y2),int(x+x4),c],img[int(y+y3),int(x+x4),c],img[int(y+y4),int(x+x4),c]]])
                
                
                mat_kernely=np.matrix([[kernel(y1,a)],[kernel(y2,a)],[kernel(y3,a)],[kernel(y4,a)]])
                
                
                matriximage[column, row, c] = np.dot(np.dot(mat_kernelx, mat_near), mat_kernely)
 
                
    cv2.imwrite('bicubicInterpolation.jpg', matriximage)

In [14]:
options = Dropdown(
    options=[('None', 0), ('Median Filter', 1), ('Averaging filter', 2),
             ('Gaussian Filter', 3), ('Unsharp Masking and Highboost Filtering', 4), ('Sobel Operators', 5), 
             ('Impulse Noise (Salt and Pepper)', 6), ('Gaussian Noise', 7), ('Histogram Eq.', 8),
             ('Fourier Transform', 9), ('Interpolation (Bicubic)', 10)],
    value=0,
    description='Select Filter:',
    layout=Layout(width='450px'))
outDropdown = Output()

slider = IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='K:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=Layout(visibility='hidden', width='475px'))
outSlider = Output()

progressBar = IntProgress(
    value=0,
    min=0,
    max=100,
    description='Loading:',
    bar_style='success', # 'success', 'info', 'warning', 'danger' or ''
    style={'bar_color': '#1cf28c'},
    orientation='horizontal',
    layout=Layout(visibility='hidden', flex='1 1 0%', width='520px'))

btns = [
    #Button(description='Apply Filter', disabled=False, icon='plus', layout=Layout(flex='1 1 0%', width='120px'), button_style='primary'),
    #Button(description='Remove Filter', disabled=False, icon='remove', layout=Layout(flex='1 1 0%', width='120px'), button_style='warning'),
 ]

resetBtn = Button(description='Reset', disabled=False, icon='trash', layout=Layout(flex='1 1 0%', width='150px'), button_style='danger')
outResetBtn = Output()
btns_box_layout = Layout(display='flex', flex_flow='row', align_items='stretch', width='30%')

btns_box = Box(children=btns, layout=btns_box_layout)

In [15]:
def work(progress, slider):
    pass

In [16]:
def on_click_resetBtn(change):
    with outResetBtn:
        pass

In [17]:
def on_slider_change(change):
    with outSlider:
        v = change['new']
        progressBar.value = 0
        unsharpMaskingAndHighboostFiltering(deepcopy(selected_img), v, progressBar)
        file = open("unsharpMaskingAndHighboostFiltering.jpg", "rb")
        image = file.read()
        image_after.value = image
        slider.layout.visibility = 'visible'

In [18]:
def on_dropdown_change(change):
    with outDropdown:
        v = change['new']
        progressBar.layout.visibility = 'visible'
        if v == 0:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            file = open("No-filter.png", "rb")
            image = file.read()
            image_after.value = image
        if v == 1:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            medianFilter(deepcopy(selected_img), progressBar)
            file = open("medianFilter.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("medianFilter.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 2:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            averaging(deepcopy(selected_img), progressBar)
            file = open("averaging.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("averaging.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 3:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            gaussianFilter(deepcopy(selected_img), 5, 0.8, progressBar)
            file = open("gaussianFilter.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("gaussianFilter.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 4:
            progressBar.value = 0
            unsharpMaskingAndHighboostFiltering(deepcopy(selected_img), 1, progressBar)
            file = open("unsharpMaskingAndHighboostFiltering.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("unsharpMaskingAndHighboostFiltering.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
            slider.layout.visibility = 'visible'
        if v == 5:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            sobelOperator(deepcopy(selected_img), progressBar)
            file = open("sobelOperator.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("sobelOperator.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 6:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            saltAndPepperNoise(deepcopy(selected_img), progressBar)
            file = open("saltAndPepperNoise.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("saltAndPepperNoise.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 7:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            gaussianNoise(deepcopy(selected_img), progressBar)
            file = open("gaussianNoise.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("gaussianNoise.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 8:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            histogramEq(deepcopy(selected_img), progressBar)
            file = open("histogramEq.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("histogramEq.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 9:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            fourierTransform(deepcopy(selected_img), progressBar)
            file = open("fourierTransform.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("fourierTransform.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image
        if v == 10:
            slider.layout.visibility = 'hidden'
            progressBar.value = 0
            bicubicInterpolation(deepcopy(selected_img), 2, -0.5, progressBar)
            file = open("bicubicInterpolation.jpg", "rb")
            image = file.read()
            new_image = cv2.imread("bicubicInterpolation.jpg")
            print("After: " , new_image.shape[0], " x ", new_image.shape[1], end="\r")
            image_after.value = image

In [19]:
ImgUpload = FileUpload(accept='image/*', multiple=False)
outUpload = Output()

display(ImgUpload, outUpload)

def on_upload_change(change):
    with outUpload:
        uploaded_file = ImgUpload.value[list(ImgUpload.value.keys())[0]]['content']

        global selected_img
        selected_img = cv2.imread(str(list(ImgUpload.value.keys())[0]), 0)
        selected_img = selected_img.reshape(selected_img.shape[0], selected_img.shape[1], 1)
        
        image_before = Image(value=uploaded_file, width=256, height=256)

        file = open("No-filter.png", "rb")
        image = file.read()
        
        global image_after
        image_after = Image(value=image, width=256, height=256)
        print("Before: " , selected_img.shape[0], " x ", selected_img.shape[1])
        display(options, HBox([image_before, image_after]), slider, progressBar, outDropdown, outSlider, outResetBtn)
        
        thread.start()
        
        
thread = threading.Thread(target=work, args=(progressBar, slider,))
ImgUpload.observe(on_upload_change, 'value')
options.observe(on_dropdown_change, 'value')
slider.observe(on_slider_change, 'value')
resetBtn.on_click(on_click_resetBtn)

FileUpload(value={}, accept='image/*', description='Upload')

Output()